In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 
from itertools import permutations
import folium


def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4, ensure_ascii=False)
    print(pretty_json)

def find_target_directory(target_dir_name):
    current_path = os.getcwd()
    while True:
        # 상위 디렉토리에서 대상 디렉토리 이름을 찾음
        parent_path, current_dir = os.path.split(current_path)
        
        if current_dir == target_dir_name:
            return current_path
        
        if parent_path == current_path:  # 최상위 디렉토리에 도달하면 종료
            break
        
        # 상위 디렉토리로 이동
        current_path = parent_path
    
    return None  # 대상 디렉토리를 찾지 못한 경우


def get_project_root_path(proejct_directory_name: str='odysseyes'):
    return  find_target_directory(target_dir_name=proejct_directory_name)


project_root_path = get_project_root_path()
print(f"프로젝트 경로: {project_root_path}")
sys.path.append(os.path.join(project_root_path, 'recommend', 'func'))

# from tmap_route_optimizer import TMAPClient, RouteOptimizer, PlaceDataManager
from tmap_client import TMAPClient
from place_data_manager import PlaceDataManager

프로젝트 경로: c:\Users\SSAFY\Desktop\workspace\odysseyes


In [2]:
load_dotenv()
api_key = os.getenv('SK_OPEN_API_KEY')

In [3]:
tmap_client = TMAPClient(api_key=api_key)
place_data_manager = PlaceDataManager()

In [4]:
start_place = '대전 서구 월평동 1216'
festival_place = '백제문화단지'

In [5]:
place_combination_list = place_data_manager.generate_place_combinations(region='부여')

In [6]:
places = place_combination_list[0]

In [27]:
def add_start_and_festival_places(places, start, festival_place):

    places = [{'name': start, 'category': '출발지', 'latitude': None, 'longitude': None}] \
        + places \
        + [{'name': festival_place, 'category': '축제장소', 'latitude': None, 'longitude': None}]
    
    return places

In [ ]:
places = add_start_and_festival_places(places=places, start=start_place, festival_place=festival_place)

In [7]:
# places = [{'name': start_place, 'category': '출발지', 'latitude': None, 'longitude': None}] \
#         + places \
#         + [{'name': festival_place, 'category': '축제장소', 'latitude': None, 'longitude': None}]

In [8]:
print_json(places)

[
    {
        "name": "대전 서구 월평동 1216",
        "category": "출발지",
        "latitude": null,
        "longitude": null
    },
    {
        "name": "무드빌리지",
        "category": "카페",
        "region": "부여",
        "latitude": 36.2928562,
        "longitude": 126.9245167,
        "방문건수순위": 1,
        "구글지도평점": 0.85,
        "데이터랩점수": 0.64,
        "방문건수순위점수": 1.0,
        "최종점수": 2.49
    },
    {
        "name": "장원막국수",
        "category": "식당",
        "region": "부여",
        "latitude": 36.289136802505,
        "longitude": 126.90872330601528,
        "방문건수순위": 1,
        "구글지도평점": 0.87,
        "데이터랩점수": 0.64,
        "방문건수순위점수": 1.0,
        "최종점수": 2.51
    },
    {
        "name": "궁남지",
        "category": "관광지",
        "region": "부여",
        "latitude": 36.26963904275301,
        "longitude": 126.91222316064965,
        "방문건수순위": 2,
        "구글지도평점": 0.92,
        "데이터랩점수": 1.0,
        "방문건수순위점수": 0.95,
        "최종점수": 2.87
    },
    {
        "name": "백제문화단지",
        

In [9]:
# 거리 및 경로 데이터를 가져오는 함수 (예시용 API 엔드포인트 사용)
def fetch_route_data(tmap_client:TMAPClient, origin, destination):
    if origin['latitude'] == None and origin['longitude'] == None:
        origin = tmap_client.get_poi(origin['name'])
    if destination['latitude'] == None and destination['longitude'] == None:
        destination = tmap_client.get_poi(destination['name'])

    response = tmap_client.get_route_data(start=origin, end=destination)
    return response  # {"distance": km, "time": min, "cost": won, "polyline": ...}


In [10]:
# 모든 장소 간 1:1 거리, 시간, 비용 데이터를 수집
routes_data = {}
for i, origin in enumerate(places):
    # print(origin)
    for j, destination in enumerate(places):
        # print(destination)
        # break
        if i != j:
            routes_data[(i, j)] = fetch_route_data(tmap_client=tmap_client, origin=origin, destination=destination)

{
    "tollgateFareOption": 16,
    "roadType": 32,
    "endX": 126.9245167,
    "endY": 36.2928562,
    "reqCoordType": "WGS84GEO",
    "startX": 127.35677043,
    "startY": 36.35496178,
    "carType": 0,
    "startName": "대전 서구 월평동 1216",
    "endName": "무드빌리지",
    "resCoordType": "WGS84GEO",
    "sort": "index"
}
{
    "tollgateFareOption": 16,
    "roadType": 32,
    "endX": 126.90872330601528,
    "endY": 36.289136802505,
    "reqCoordType": "WGS84GEO",
    "startX": 127.35677043,
    "startY": 36.35496178,
    "carType": 0,
    "startName": "대전 서구 월평동 1216",
    "endName": "장원막국수",
    "resCoordType": "WGS84GEO",
    "sort": "index"
}
{
    "tollgateFareOption": 16,
    "roadType": 32,
    "endX": 126.91222316064965,
    "endY": 36.26963904275301,
    "reqCoordType": "WGS84GEO",
    "startX": 127.35677043,
    "startY": 36.35496178,
    "carType": 0,
    "startName": "대전 서구 월평동 1216",
    "endName": "궁남지",
    "resCoordType": "WGS84GEO",
    "sort": "index"
}
{
    "tollgateFare

In [11]:
print(len(routes_data))

20


In [12]:
print_json(routes_data[(0, 1)])

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    127.35686486460436,
                    36.35506787940311
                ]
            },
            "properties": {
                "totalDistance": 69080,
                "totalTime": 3682,
                "totalFare": 2700,
                "taxiFare": 69160,
                "index": 0,
                "pointIndex": 0,
                "name": "",
                "description": "월평새뜸로4번길 을 따라  방면으로 24m 이동",
                "nextRoadName": "월평새뜸로4번길",
                "turnType": 200,
                "pointType": "S"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [
                    [
                        127.35686486460436,
                        36.35506787940311

In [13]:
from sklearn.preprocessing import MinMaxScaler

def get_scaled_properties(routes: dict):
    """
    TMap API를 통해 얻어온 경로 정보에서 거리, 소요 시간, 비용을 정규화하고,
    각 경로의 상대적 점수를 계산하여 추가하는 함수.

    Args:
        routes (dict): 각 경로의 정보를 담고 있는 데이터.

    Returns:
        dict: 정규화된 점수 정보가 추가된 경로 데이터.
    """
    # 모든 경로의 'totalDistance', 'totalTime', 'totalFare'만 추출
    selected_properties_data = [
        {
            'totalDistance': route['features'][0]['properties']['totalDistance'],
            'totalTime': route['features'][0]['properties']['totalTime'],
            'totalFare': route['features'][0]['properties']['totalFare']
        }
        for route in routes.values()
    ]

    # DataFrame 변환 및 정규화
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(pd.DataFrame(selected_properties_data))

    # 정규화된 데이터를 바탕으로 scaledProperties 생성
    scaled_properties = [
        {
            'scaledDistance': round(1 - scaled[0], 2),
            'scaledTime': round(1 - scaled[1], 2),
            'scaledFare': round(1 - scaled[2], 2),
            'scaledDTFScore': round(1 - scaled[0] + 1 - scaled[1] + 1 - scaled[2], 2)
        }
        for scaled in scaled_data
    ]

    # 원본 데이터에 정규화된 점수 추가
    for route, scaled_property in zip(routes.values(), scaled_properties):
        route['features'][0]['properties']['scaledProperties'] = scaled_property

    return routes


In [14]:
# from sklearn.preprocessing import MinMaxScaler

# def get_scaled_properties(routes:dict):
#     # routes = list(routes.values())
#     route_dict = []
#     for _, v in routes.items():    
#         route_dict.append(v)
 
#     all_features_list = [route['features'][0] for route in route_dict]  # 모든 1대1 경로들의 features 리스트
#     all_properties_list = [features['properties'] for features in all_features_list]  # 모든 1대1 경로들의 경로 요약 정보 

#     # 경로들의 전체 거리, 소요 시간, 비용만 추출한 리스트 
#     selected_properties_data = [
#         {
#             'totalDistance': properties['totalDistance'],
#             'totalTime': properties['totalTime'],
#             'totalFare': properties['totalFare'] 
#         }
#         for properties in all_properties_list
#     ]

#     scaler = MinMaxScaler()

#     scaled_selected_properties_data = scaler.fit_transform(pd.DataFrame(selected_properties_data))
#     scaled_scores = []
#     for i in range(len(scaled_selected_properties_data[0])):
#         scaled_score = []
#         for j in range(len(scaled_selected_properties_data)):
#             scaled_score.append(round(float(scaled_selected_properties_data[j][i]), 3))
#         scaled_scores.append(scaled_score)
    
#     scaledProperties = []
#     totalRouteScores = []
#     for i in range(len(scaled_scores[0])):
#         scaledProperty = {
#             'scaledDistance': round(1 - scaled_scores[0][i], 2),
#             'scaledTime': round(1 - scaled_scores[1][i], 2),
#             'scaledFare': round(1 - scaled_scores[2][i], 2),
#         }
        
#         scaledDTFScore = round(sum(scaledProperty.values()), 2)
#         scaledProperty['scaledDTFScore'] = scaledDTFScore
#         scaledProperties.append(scaledProperty)

#     for i, route in enumerate(routes.values()):
#         print(route)
#         print(scaledProperties[i])
#         _features = route['features'][0]
#         _properties = _features['properties']
#         _properties['scaledProperties'] = scaledProperties[i]

#     return routes


In [15]:
routes_data_with_scaeld_properties = get_scaled_properties(routes=routes_data)

In [16]:
# 각 경로에 대한 점수를 계산하는 함수
def calculate_route_score(route, routes_data, weight_distance=0.4, weight_time=0.4, weight_fare=0.2):
    # print(route)
    score = 0
    for i in range(len(route) - 1):
        route_info = routes_data[(route[i], route[i + 1])]
        # print(0000)
        # print(type(route_info))
        # route_info = get_scaled_properties(route_info)
        route_features = route_info['features'][0]
        # print_json(route_info)
        route_properties = route_features['properties']
        route_scaled_properties = route_properties['scaledProperties']
        # print(route_info_scaled_properties)
        score += route_scaled_properties['scaledDTFScore']
    return score

In [17]:
# 출발지에서 시작하여 모든 장소를 방문 후 출발지로 돌아오는 최적 경로 탐색
def find_optimal_route(places, routes_data):
    num_places = len(places)
    all_routes = list(permutations(range(1, num_places)))  # 출발지를 제외한 경유지들의 순열 생성
    best_score = 0
    best_route = None

    print(f'출발지 제외 경유지 순열: {list(all_routes)}')   
    # print(f"routes_data:")
    # print_json(routes_data)

    for route in all_routes:
        # print(route)
        route = (0,) + route + (0,)  # 출발지(0)를 추가해 순환 경로 형성
        print(route)
        score = round(calculate_route_score(route, routes_data), 4)
        print(f'{route}: {score}')
        if score > best_score:
            best_score = score
            best_route = route
            
    return best_route, best_score

In [18]:
# 최적 경로 탐색 수행
best_route, best_score = find_optimal_route(places=places, routes_data=routes_data_with_scaeld_properties)
print("최적 경로:", best_route)
print("최적 점수:", best_score)

출발지 제외 경유지 순열: [(1, 2, 3, 4), (1, 2, 4, 3), (1, 3, 2, 4), (1, 3, 4, 2), (1, 4, 2, 3), (1, 4, 3, 2), (2, 1, 3, 4), (2, 1, 4, 3), (2, 3, 1, 4), (2, 3, 4, 1), (2, 4, 1, 3), (2, 4, 3, 1), (3, 1, 2, 4), (3, 1, 4, 2), (3, 2, 1, 4), (3, 2, 4, 1), (3, 4, 1, 2), (3, 4, 2, 1), (4, 1, 2, 3), (4, 1, 3, 2), (4, 2, 1, 3), (4, 2, 3, 1), (4, 3, 1, 2), (4, 3, 2, 1)]
(0, 1, 2, 3, 4, 0)
(0, 1, 2, 3, 4, 0): 9.57
(0, 1, 2, 4, 3, 0)
(0, 1, 2, 4, 3, 0): 9.26
(0, 1, 3, 2, 4, 0)
(0, 1, 3, 2, 4, 0): 9.59
(0, 1, 3, 4, 2, 0)
(0, 1, 3, 4, 2, 0): 9.15
(0, 1, 4, 2, 3, 0)
(0, 1, 4, 2, 3, 0): 9.44
(0, 1, 4, 3, 2, 0)
(0, 1, 4, 3, 2, 0): 9.35
(0, 2, 1, 3, 4, 0)
(0, 2, 1, 3, 4, 0): 9.42
(0, 2, 1, 4, 3, 0)
(0, 2, 1, 4, 3, 0): 9.29
(0, 2, 3, 1, 4, 0)
(0, 2, 3, 1, 4, 0): 9.6
(0, 2, 3, 4, 1, 0)
(0, 2, 3, 4, 1, 0): 9.55
(0, 2, 4, 1, 3, 0)
(0, 2, 4, 1, 3, 0): 9.27
(0, 2, 4, 3, 1, 0)
(0, 2, 4, 3, 1, 0): 9.39
(0, 3, 1, 2, 4, 0)
(0, 3, 1, 2, 4, 0): 9.27
(0, 3, 1, 4, 2, 0)
(0, 3, 1, 4, 2, 0): 9.03
(0, 3, 2, 1, 4, 0)
(0, 3, 2, 1, 4

In [19]:
# 장소 인덱스로 나오는 경유지 최적화 순서를 tmap_client.get_route_data()로 넘기기 위한 데이터 생성
# def get_route_data(self, start: dict, end: dict, passList: list=None):

start = places[0]
end = places[0]
passList = places[1:]

print_json(start)
print_json(end)
print_json(passList)

{
    "name": "대전 서구 월평동 1216",
    "category": "출발지",
    "latitude": null,
    "longitude": null
}
{
    "name": "대전 서구 월평동 1216",
    "category": "출발지",
    "latitude": null,
    "longitude": null
}
[
    {
        "name": "무드빌리지",
        "category": "카페",
        "region": "부여",
        "latitude": 36.2928562,
        "longitude": 126.9245167,
        "방문건수순위": 1,
        "구글지도평점": 0.85,
        "데이터랩점수": 0.64,
        "방문건수순위점수": 1.0,
        "최종점수": 2.49
    },
    {
        "name": "장원막국수",
        "category": "식당",
        "region": "부여",
        "latitude": 36.289136802505,
        "longitude": 126.90872330601528,
        "방문건수순위": 1,
        "구글지도평점": 0.87,
        "데이터랩점수": 0.64,
        "방문건수순위점수": 1.0,
        "최종점수": 2.51
    },
    {
        "name": "궁남지",
        "category": "관광지",
        "region": "부여",
        "latitude": 36.26963904275301,
        "longitude": 126.91222316064965,
        "방문건수순위": 2,
        "구글지도평점": 0.92,
        "데이터랩점수": 1.0,
        "방문건수순위점수"

In [20]:
route = tmap_client.get_route_data(start=start, end=end, passList=passList)

{
    "tollgateFareOption": 16,
    "roadType": 32,
    "endX": 127.35677043,
    "endY": 36.35496178,
    "reqCoordType": "WGS84GEO",
    "startX": 127.35677043,
    "startY": 36.35496178,
    "carType": 0,
    "startName": "대전 서구 월평동 1216",
    "endName": "대전 서구 월평동 1216",
    "resCoordType": "WGS84GEO",
    "sort": "index",
    "passList": "126.9245167,36.2928562_126.90872330601528,36.289136802505_126.91222316064965,36.26963904275301_126.90670093,36.30662608"
}


In [21]:
print_json(route)

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    127.35686486460436,
                    36.35506787940311
                ]
            },
            "properties": {
                "totalDistance": 152241,
                "totalTime": 9196,
                "totalFare": 5900,
                "taxiFare": 151470,
                "index": 0,
                "pointIndex": 0,
                "name": "",
                "description": "월평새뜸로4번길 을 따라  방면으로 24m 이동",
                "nextRoadName": "월평새뜸로4번길",
                "turnType": 200,
                "pointType": "S"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [
                    [
                        127.35686486460436,
                        36.355067879403